In [1]:
import string
import re
import random
import gensim
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [3]:
#/Users/kosekitomohiro/Downloads/entity_vector
model_dir = '/Users/kosekitomohiro/Downloads/entity_vector/entity_vector.model.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(model_dir, binary=True)
embedding_vector = torch.from_numpy(model.vectors)

In [4]:
SOS_token = 0 #start of string
EOS_token = 1
hidden_size = 256
MAX_LENGTH = 30 #文章の最大単語数(句読点を含む)

In [5]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, text_embedding_vectors, dropout_p):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        # input_size：辞書の単語種類数、 hidden_size：分散表現の次元数
        self.embedding = nn.Embedding.from_pretrained(
        embeddings=text_embedding_vectors, freeze=True)
        self.emb_dim = text_embedding_vectors.shape[1]

        #層を増やす実験
        #self.gru1 = nn.GRU(text_embedding_vectors.shape[1], hidden_size)
        self.lstm = nn.LSTM(text_embedding_vectors.shape[1],
                            hidden_size, dropout=dropout_p) #[200, 256]

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.lstm(output, hidden)
        #output, hidden2 = self.gru2(output, hidden2)
        #出力と隠れ状態
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))
    
    
    
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, text_embedding_vectors, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding.from_pretrained(embeddings=text_embedding_vectors, freeze=True)
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.attn = nn.Linear(self.hidden_size + text_embedding_vectors.shape[1], self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size + text_embedding_vectors.shape[1], self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        #self.gru1 = nn.GRU(self.hidden_size, self.hidden_size)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size, dropout=dropout_p)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        #input:[1], hidden:[1,1,256], encoder_outputs:[10,256]
        embedded = self.embedding(input).view(1, 1, -1) #[1, 1, 256]
        embedded = self.dropout(embedded)

        #加法注意によりattention_weightを求める。
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0][0]), 1)), dim=1)

        #weightとエンコーダのアウトプットを用いる
        #bmmは各バッチごとに内積を出せる
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1) #[1, 512]
        output = self.attn_combine(output).unsqueeze(0) #[1, 1, 256]

        output = F.relu(output) #[1, 1, 256]
        output, hidden = self.lstm(output, hidden) #[1, 1, 256], ([1, 1, 256], [1, 1, 256])

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

In [12]:
import pickle
# index→sentence 
input_lang_index2word = pickle.load(open('/Users/kosekitomohiro/Downloads/model/input_lang_index2word0517.pkl', 'rb'))
input_lang_word2index = pickle.load(open('/Users/kosekitomohiro/Downloads/model/input_lang_word2index0517.pkl', 'rb'))
output_lang_index2word = pickle.load(open('/Users/kosekitomohiro/Downloads/model/output_lang_index2word0517.pkl', 'rb'))
output_lang_word2index = pickle.load(open('/Users/kosekitomohiro/Downloads/model/output_lang_word2index0517.pkl', 'rb'))

In [56]:
#model_path = '/Users/kosekitomohiro/Downloads/model/encoder0517.pth'
encoder1 = EncoderRNN(len(input_lang_index2word ), hidden_size,
                      embedding_vector, dropout_p=0.1)
encoder1.load_state_dict(torch.load(
    '/Users/kosekitomohiro/Downloads/model/encoder0517.pth')) #モデルにパラメータを当てはめる
encoder1.eval()

attn_decoder1 = AttnDecoderRNN(hidden_size,  embedding_vector,
                               len(output_lang_index2word), dropout_p=0.1).to(device)
attn_decoder1.load_state_dict(torch.load(
    '/Users/kosekitomohiro/Downloads/model/decoder0517.pth')) #モデルにパラメータを当てはめる
attn_decoder1.eval()

AttnDecoderRNN(
  (embedding): Embedding(1015474, 200)
  (attn): Linear(in_features=456, out_features=30, bias=True)
  (attn_combine): Linear(in_features=456, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (lstm): LSTM(256, 256, dropout=0.1)
  (out): Linear(in_features=256, out_features=6484, bias=True)
)

In [39]:
import MeCab

mecab = MeCab.Tagger('-Owakati')
def normalizeString(s):
    #s = unicodeToAscii(s.lower().strip())
    s = re.sub('\r', '', s)
    s = re.sub('\n', '', s)
    s = re.sub(' ', ' ', s)
    s = mecab.parse(s)[:-2] #分かち書き
    #入力データを反転させる
    s = ' '.join(s.split(' ')[::-1])
    return s #sentence
normalizeString('退屈に耐えるのです')

'です の 耐える に 退屈'

In [22]:
def indexesFromSentence(input_lang, sentence):
    #各文を単語indexのならびに
    return [input_lang[word] for word in sentence.split()]


def tensorFromSentence(input_lang, sentence):
    indexes = indexesFromSentence(input_lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long).view(-1, 1) #[[2], [25], [9]....]の形に


def tensorsFromPair(input_lang, sentence):
    #pairは各文(0がフランス)
    input_tensor = tensorFromSentence(input_lang, sentence)
    #target_tensor = tensorFromSentence(output_lang, pair[1])
    return input_tensor

In [29]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang_word2index, sentence)
        print(input_tensor)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]])  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<eos>')
                break
            else:
                decoded_words.append(output_lang_index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [69]:
decoded_words, decoder_attentions = evaluate(encoder1, attn_decoder1, normalizeString('こんにちは'), max_length=MAX_LENGTH)
print(''.join(decoded_words))

tensor([[298],
        [  1]])
こんにちは。声かけてくれるのを待ってたんだ。<eos>
